# PYTORCH PROFILER

This recipe explains how to use PyTorch profiler and measure the time and memory consumption of the model’s operators.

## Introduction

PyTorch includes a simple profiler API that is useful when user needs to determine the most expensive operators in the model.

In this recipe, we will use a simple Resnet model to demonstrate how to use profiler to analyze model performance.

## Steps

1. Import all necessary libraries
2. Instantiate a simple Resnet model
3. Using profiler to analyze execution time
4. Using profiler to analyze memory consumption
5. Using tracing functionality
6. Examining stack traces
7. Visualizing data as a flamegraph
8. Using profiler to analyze long-running jobs

### 1. Import all necessary libraries
In this recipe we will use ```torch```, ```torchvision.models``` and ```profiler``` modules:

In [2]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

### 2. Instantiate a simple Resnet model

Let’s create an instance of a Resnet model and prepare an input for it:

In [2]:
model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)


### 3. Using profiler to analyze execution time

PyTorch profiler is enabled through the context manager and accepts a number of parameters, some of the most useful are:

- ```activities``` - a list of activities to profile:
    - ```ProfilerActivity.CPU``` - PyTorch operators, TorchScript functions and user-defined code labels (see ```record_function``` below);
    - ```ProfilerActivity.CUDA``` - on-device CUDA kernels;
- ```record_shapes``` - whether to record shapes of the operator inputs;
- ```profile_memory``` - whether to report amount of memory consumed by model’s Tensors;
- ```use_cuda``` - whether to measure execution time of CUDA kernels.

Note: when using CUDA, profiler also shows the runtime CUDA events occuring on the host.

Let’s see how we can use profiler to analyze the execution time:

In [3]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory=True) as prof:
    with record_function("model_inference"):
        model(inputs)

Note that we can use ```record_function``` context manager to label arbitrary code ranges with user provided names (```model_inference``` is used as a label in the example above).

Profiler allows one to check which operators were called during the execution of a code range wrapped with a profiler context manager. If multiple profiler ranges are active at the same time (e.g. in parallel PyTorch threads), each profiling context manager tracks only the operators of its corresponding range. Profiler also automatically profiles the async tasks launched with ```torch.jit._fork``` and (in case of a backward pass) the backward pass operators launched with ```backward()``` call.

Let’s print out the stats for the execution above:

In [4]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         1.89%       7.389ms        99.41%     388.669ms     388.669ms          -4 b    -106.30 Mb             1  
                     aten::conv2d         0.63%       2.474ms        82.83%     323.844ms      16.192ms      47.37 Mb           0 b            20  
                aten::convolution         0.13%     515.000us        82.20%     321.370ms      16.069ms      47.37 Mb           0 b            20  
               aten::_convolution         0.17%     652.000us        82.07%     320.855ms      16.043ms      47.

Here we see that, as expected, most of the time is spent in convolution (and specifically in ```mkldnn_convolution``` for PyTorch compiled with MKL-DNN support). Note the difference between self cpu time and cpu time - operators can call other operators, self cpu time exludes time spent in children operator calls, while total cpu time includes it. You can choose to sort by the self cpu time by passing ```sort_by="self_cpu_time_total"``` into the ```table``` call.

To get a finer granularity of results and include operator input shapes, pass ```group_by_input_shape=True``` (note: this requires running the profiler with ```record_shapes=True```):

In [5]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                  model_inference         1.89%       7.389ms        99.41%     388.669ms     388.669ms          -4 b    -106.30 Mb             1                                                                                []  
                     aten::conv2d         0.58%       2.270ms        34.79%     

Note the occurence of ```aten::convolution``` twice with different input shapes.

Profiler can also be used to analyze performance of models executed on GPUs: (Note: the first use of CUDA profiling may bring an extra overhead.)

In [3]:
model = models.resnet18().cuda()
inputs = torch.randn(5, 3, 224, 224).cuda()

with profile(activities=[
        ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))



Note the occurence of on-device kernels in the output (e.g. ```sgemm_32x32x32_NN```).